In [1]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd

In [2]:
url = 'https://www.california-demographics.com/zip_codes_by_population'
response = requests.get(url, 'html.parser')
page_soup = soup(response.text)
population = []
for tr in page_soup.find_all('tr')[1:]:
    tds = tr.find_all('td')
    if len(tds[1].text.strip()) == 5:
        population.append([tds[1].text.strip(), tds[2].text.strip()])

IndexError: list index out of range

In [3]:
populationdf = pd.DataFrame(population)
populationdf.columns = ['zip', 'population']
populationdf

,zip,population
0,90011,"108,051"
1,90650,"106,404"
2,91331,"105,696"
3,90201,"102,878"
4,92335,"99,791"
5,90250,"97,371"
6,91342,"96,177"
7,90805,"96,069"
8,90280,"95,420"
9,90044,"94,680"


In [8]:
starbucks = pd.read_csv('directory.csv')
starbucks = starbucks[['Brand','City','State/Province', 'Postcode']]
starbucks.columns = ['brand', 'city', 'state', 'zip']
starbucks = starbucks[starbucks['city']=='Los Angeles']
starbucksdf = pd.DataFrame(starbucks['zip'].str.slice(0,5,1).value_counts())
starbucksdf.reset_index(level=0, inplace=True)
starbucksdf.columns = ['zip', 'store_count']
starbucksdf

,zip,store_count
0,90045,20
1,90036,9
2,90064,7
3,90012,6
4,90071,6
5,90015,5
6,90041,4
7,90048,4
8,90046,4
9,90028,4


In [9]:
url = 'http://www.laalmanac.com/employment/em12c.php'
response = requests.get(url, 'html.parser')
page_soup = soup(response.text)
income = []
for tr in page_soup.find_all('tr')[1:]:
    tds = tr.find_all('td')
    income.append([tds[0].text.strip(), tds[1].text.strip(), tds[2].text.strip()])

In [10]:
incomedf = pd.DataFrame(income)
incomedf.columns = ['zip', 'community', 'median_income']
demographicsdf = incomedf.merge(populationdf, on='zip')

In [11]:
demographicsdf.head()

,zip,community,median_income,population
0,90001,"Los Angeles (South Los Angeles), Florence-Graham","$35,660","58,738"
1,90002,"Los Angeles (Southeast Los Angeles, Watts)","$34,000","52,856"
2,90003,"Los Angeles (South Los Angeles, Southeast Los ...","$34,397","70,490"
3,90004,"Los Angeles (Hancock Park, Rampart Village, Vi...","$46,581","62,733"
4,90005,"Los Angeles (Hancock Park, Koreatown, Wilshire...","$32,461","39,562"


In [12]:
finaldf = demographicsdf.merge(starbucksdf, on='zip')

In [ ]:
finaldf